# DeepAF minimal demo
Run top to bottom.

In [ ]:
import os, shutil
proj = '/content/deepaf'
if os.path.exists(proj): shutil.rmtree(proj)
shutil.copytree('/mnt/data/deepaf', proj)
print('Project ready at', proj)
!ls -R /content/deepaf


In [ ]:
!pip -q install torch==2.3.1 torchvision==0.18.1 --index-url https://download.pytorch.org/whl/cu121
!pip -q install opencv-python matplotlib tqdm ultralytics scikit-image


In [ ]:
!python -m src.data.simulate_defocus --out /content/deepaf/data_synth --n 1200 --size 256


In [ ]:
!python -m src.models.train_step_estimator --data /content/deepaf/data_synth --epochs 2 --batch 64 --size 256


In [ ]:
!python -m src.models.train_focus_discriminator --data /content/deepaf/data_synth --epochs 2 --batch 64 --size 256


In [ ]:
import cv2 as cv, numpy as np, os
os.makedirs('/content/deepaf/assets', exist_ok=True)
w,h = 640,360
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter('/content/deepaf/assets/sample.mp4', fourcc, 20.0, (w,h))
for t in range(200):
    frame = np.zeros((h,w,3), np.uint8)
    cx = int(100 + 400*(t/200.0))
    cy = int(120 + 60*np.sin(t/12.0))
    cv.circle(frame, (cx,cy), 30, (0,255,0), -1)
    out.write(frame)
out.release()
!python -m src.af.inference_controller --video /content/deepaf/assets/sample.mp4 --use_yolo 0 --step_w runs/step_estimator.pt --disc_w runs/focus_discriminator.pt
